In [ ]:
import pandas as pd
import json

# Load your dataset
file_path = 'Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv'
df = pd.read_csv(file_path)

# Transform the dataset into the required format
jsonl_data = []
for _, row in df.iterrows():
    entry = {
        "messages": [
            {"role": "system", "content": "You are a helpful e-commerce customer service assistant. Provide clear, accurate and stylish answers and relevant product recommendations."},
            {"role": "user", "content": row["instruction"]},
            {"role": "assistant", "content": row["response"]}
        ]
    }
    jsonl_data.append(entry)

# Save as JSONL
output_file = 'customer_service_fine_tune_data.jsonl'
with open(output_file, 'w') as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + '\n')

print(f"Dataset saved to {output_file}")


Dataset saved to customer_service_fine_tune_data.jsonl


In [ ]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [ ]:
pip install tiktoken

In [ ]:
data_path = "customer_service_fine_tune_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 26872
First example:
{'role': 'system', 'content': 'You are a helpful e-commerce customer service assistant. Provide clear, accurate and stylish answers and relevant product recommendations.'}
{'role': 'user', 'content': 'question about cancelling order {{Order Number}}'}
{'role': 'assistant', 'content': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}


In [ ]:
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 55, 527
mean / median: 172.25520988389403, 151.5
p5 / p95: 113.0, 261.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 12, 478
mean / median: 125.24006400714498, 104.0
p5 / p95: 67.0, 214.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~4628842 tokens that will be charged for during training
By default, you'll train for 1 epochs on this dataset
By default, you'll be charged for ~4628842 tokens


In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

client.files.create(
  file=open("customer_service_fine_tune_data.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-Kwqa8M0tR7cMCSbWgDRXZuHd",
  model="gpt-4o-mini-2024-07-18",
   hyperparameters={
    "n_epochs":3
  }
)

FineTuningJob(id='ftjob-BZwIipxudjU2dXTo0AXsvCKI', created_at=1724188804, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-Hh0nIE9drk4sJaBIhtI2Lkj9', result_files=[], seed=431097433, status='validating_files', trained_tokens=None, training_file='file-Kwqa8M0tR7cMCSbWgDRXZuHd', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")


# Initialize conversation history

system_prompt=""" You are a specialized e-commerce customer service assistant. Your primary role is to assist with questions related to products, orders, shipping, returns, and customer support inquiries. Please avoid answering questions unrelated to e-commerce or customer service. Provide clear, accurate, and stylish answers while offering relevant product recommendations."""
conversation_history = [
    {"role": "system", "content": system_prompt},
]



In [ ]:

UserMessage="who r u ?"
conversation_history.append({"role": "user", "content":UserMessage })

# Generate assistant's response
completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::9ySAnm7D",
    messages=conversation_history
)

response = completion.choices[0].message.content

# Update conversation history
conversation_history.append({"role": "assistant", "content": response})
for message in conversation_history[1:]:
    print(message['content'])


Hi boss
Thank you for reaching out! While I appreciate the friendly greeting, I would like to remind you that I am here to assist you with your inquiries related to our company. Please let me know if you have any questions or concerns, and I'll be more than happy to help you.
who r u ?
I appreciate your curiosity! My purpose is to assist you in any way I can. I am here to provide you with information, support, and guidance related to our products and services. Your satisfaction and well-being are of utmost importance and I am dedicated to making your experience with us exceptional. Please let me know if there is anything specific you would like assistance with.
